# Imports

In [1]:
import requests
import pandas as pd
import re

# Bartender Examiner 
https://github.com/jdmartinho/BartenderExaminer/blob/master/data/cocktails.csv

In [310]:
df = pd.read_csv('bartenderexam.csv', dtype={'Glass': str, 'Other Alcohols': str, 'Mixes': str, 'Garnish': str})
df.head(20)

,Name,Glass,Main Alcohol,Other Alcohols,Mixes,Garnish
0,Highball,NaN,Bourbon,NaN,Gingerale,NaN
1,Scotch and Soda,NaN,Scotch,NaN,Soda,NaN
2,7 & 7,NaN,Seagram 7,NaN,7up,NaN
3,Whiskey and Water,NaN,Whiskey,NaN,Water,NaN
4,Rum and Coke,NaN,Light Rum,NaN,Coke,NaN
5,Cuba Libre,NaN,Light Rum,NaN,Coke,Squeeze and garnish with lime
6,Gin and Tonic,NaN,Gin,NaN,Tonic,Lime
7,Presbyterian,NaN,Bourbon,NaN,1/2 Soda 1/2 Gingerale,NaN
8,Gin Rickey,NaN,Gin,NaN,Soda,Lime
9,Old Fashioned,NaN,Bourbon,NaN,Soda,"1/2oz simple syrup or sugar, 2 dashes bitters,..."


In [311]:
df[df['Main Alcohol'].isna()]

,Name,Glass,Main Alcohol,Other Alcohols,Mixes,Garnish


In [312]:
# Drop rows without at least a main alcohol
df.dropna(subset=['Main Alcohol'], inplace=True)

In [313]:
# Fill nulls in other rows with empty strings
df.fillna({'Glass': '', 'Other Alcohols': '', 'Mixes': '', 'Garnish': ''}, inplace=True) #df[df['Other Alcohols'].isna()]

In [314]:
df.head()

,Name,Glass,Main Alcohol,Other Alcohols,Mixes,Garnish
0,Highball,,Bourbon,,Gingerale,
1,Scotch and Soda,,Scotch,,Soda,
2,7 & 7,,Seagram 7,,7up,
3,Whiskey and Water,,Whiskey,,Water,
4,Rum and Coke,,Light Rum,,Coke,


### Clean Main Alcohol Field

In [315]:
df['Main Alcohol']#[80:95] #.map(lambda x: x.split())

0                                                Bourbon
1                                                 Scotch
2                                              Seagram 7
3                                                Whiskey
4                                              Light Rum
5                                              Light Rum
6                                                    Gin
7                                                Bourbon
8                                                    Gin
9                                                Bourbon
10                                                 Vodka
11                                               Tequila
12                                                 Vodka
13                                                 Vodka
14                                                 Vodka
15                                                 Vodka
16                                               Tequila
17                             

### Remove measurement text

In [316]:
#-------------------Remove measurement text
# # 1/2, 1/2oz, 1oz, 1-, 2 cups
p = "(\d-\d/)?(\d-\d/\d)?(\d/\d)?(\doz)?(cups)?(cans)?(z)?(oz)?"
pattern = re.compile(p)

# remove measurement text from strings
df['Main Alcohol'] = df['Main Alcohol'].map(lambda x: re.sub(pattern, '', x))

In [317]:
df['Main Alcohol']#[80:95] #.map(lambda x: x.split())

0                                                Bourbon
1                                                 Scotch
2                                              Seagram 7
3                                                Whiskey
4                                              Light Rum
5                                              Light Rum
6                                                    Gin
7                                                Bourbon
8                                                    Gin
9                                                Bourbon
10                                                 Vodka
11                                               Tequila
12                                                 Vodka
13                                                 Vodka
14                                                 Vodka
15                                                 Vodka
16                                               Tequila
17                             

### Separate out the recipes with two choices of alcohol


In [318]:
twochoice = df[(df['Main Alcohol'].map(lambda x: (re.search('\w+/\w+', x)) is not None))]
# Prepare two dataframes to create new recipes for each of the choices
df1 = twochoice.copy()
df2 = twochoice.copy()
df1

,Name,Glass,Main Alcohol,Other Alcohols,Mixes,Garnish
39,Martini,,Gin/Vodka,6 drops Dry Vermouth,,Olive
40,Dry Martini,,Gin/Vodka,3 drops Dry Vermouth,,Olive
41,Extra Dry Martini,,Gin/Vodka,0-1 drops Dry Vermouth,,Olive
42,Gibson,,Gin/Vodka,6 drops Dry Vermouth,,Onion
115,Liquid Cocaine,,"Jagermeister, Rumplemint/Goldschlager, Bacardi...",,,


In [319]:
# Make sure we can find the pattern correctly
re.findall('\w+/\w+', df['Main Alcohol'][42])

['Gin/Vodka']

In [320]:
# Use the first alcohol choice in df1, and the second choice in df2
df1['Main Alcohol'] = df1['Main Alcohol'].apply(lambda x: re.sub( re.findall('\w+/\w+',x)[0], re.findall('\w+/\w+',x)[0].split("/")[0], x))
df2['Main Alcohol'] = df2['Main Alcohol'].apply(lambda x: re.sub(re.findall('\w+/\w+',x)[0], re.findall('\w+/\w+',x)[0].split("/")[1], x))


In [321]:
df1

,Name,Glass,Main Alcohol,Other Alcohols,Mixes,Garnish
39,Martini,,Gin,6 drops Dry Vermouth,,Olive
40,Dry Martini,,Gin,3 drops Dry Vermouth,,Olive
41,Extra Dry Martini,,Gin,0-1 drops Dry Vermouth,,Olive
42,Gibson,,Gin,6 drops Dry Vermouth,,Onion
115,Liquid Cocaine,,"Jagermeister, Rumplemint, Bacardi 151",,,


In [322]:
df2

,Name,Glass,Main Alcohol,Other Alcohols,Mixes,Garnish
39,Martini,,Vodka,6 drops Dry Vermouth,,Olive
40,Dry Martini,,Vodka,3 drops Dry Vermouth,,Olive
41,Extra Dry Martini,,Vodka,0-1 drops Dry Vermouth,,Olive
42,Gibson,,Vodka,6 drops Dry Vermouth,,Onion
115,Liquid Cocaine,,"Jagermeister, Goldschlager, Bacardi 151",,,


In [323]:
# Concatenate all the split up rows
new_rows = df1.append(df2)
# Drop the rows in the original dataframe with the /'s
df.drop(twochoice.index, inplace=True)
# Append the new rows with the split up alcohols to the full dataframe
df = df.append(new_rows, ignore_index=True)

In [324]:
df.shape

(131, 6)

In [325]:
# remove measurement words like cup, can, etc
remove = ['cup', 'cups','can','cans','zoz']
df['Main Alcohol'] = df['Main Alcohol'].apply(lambda x: ' '.join([i for i in x.lower().split() if i not in remove]))

In [326]:
df['Main Alcohol'] = df['Main Alcohol'].apply(lambda x: re.sub("'",'',x) )

In [327]:
df['Main Alcohol']

0                                                bourbon
1                                                 scotch
2                                              seagram 7
3                                                whiskey
4                                              light rum
5                                              light rum
6                                                    gin
7                                                bourbon
8                                                    gin
9                                                bourbon
10                                                 vodka
11                                               tequila
12                                                 vodka
13                                                 vodka
14                                                 vodka
15                                                 vodka
16                                               tequila
17                             

### Clean 'Other Alcohols' column
Can repeat many steps from Main Alcohol column

In [328]:
df['Other Alcohols']

0                                
1                                
2                                
3                                
4                                
5                                
6                                
7                                
8                                
9                                
10                               
11                               
12                               
13                               
14                               
15                 1/2oz Galliano
16                               
17                               
18                               
19                               
20                               
21           1/2oz Peach Schnapps
22                               
23         1/2oz Southern Comfort
24                               
25            1/2oz Midori on top
26                               
27             1/2oz Vodka on top
28     1/2oz Brown Creme de Cacao
29     1/2oz W

In [329]:
#-------------------Remove measurement text
# # 1/2, 1/2oz, 1oz, 1-, 2 cups
p = "(\d-\d)?(\d-\d/\d)?(\d/\d)?(\doz)?(\d drops)?(drops)?(cups)?(cans)?(z)?(oz)?"
pattern = re.compile(p)

# remove measurement text from strings
df['Other Alcohols'] = df['Other Alcohols'].map(lambda x: re.sub(pattern, '', x))

In [330]:
df['Other Alcohols']

0                           
1                           
2                           
3                           
4                           
5                           
6                           
7                           
8                           
9                           
10                          
11                          
12                          
13                          
14                          
15                  Galliano
16                          
17                          
18                          
19                          
20                          
21            Peach Schnapps
22                          
23          Southern Comfort
24                          
25             Midori on top
26                          
27              Vodka on top
28      Brown Creme de Cacao
29      White Creme de Cacao
               ...          
101                         
102                         
103                         
104           

In [331]:
# Remove undesired words and phrases
remove = ['on', 'top', 'No']
df['Other Alcohols'] = df['Other Alcohols'].apply(lambda x: ' '.join([i for i in x.lower().split() if i not in remove]))

In [332]:
df['Other Alcohols'][30:90]

30                       white creme de cacao
31                                 triple-sec
32                       white creme de cacao
33                       white creme de cacao
34                                           
35                                           
36                                     kahlua
37                                           
38                                  vermouths
39                             sweet vermouth
40                               dry vermouth
41               dry vermouth, sweet vermouth
42                             sweet vermouth
43                               dry vermouth
44               dry vermouth, sweet vermouth
45                               dry vermouth
46                                benedictine
47                              grand marnier
48                                     kahlua
49                                     kahlua
50                                     kahlua
51                                

### Clean Mixes columns

In [333]:
df['Mixes']

0                          Gingerale
1                               Soda
2                                7up
3                              Water
4                               Coke
5                               Coke
6                              Tonic
7             1/2 Soda 1/2 Gingerale
8                               Soda
9                               Soda
10                      Tomato Juice
11                      Tomato Juice
12                  Grapefruit Juice
13                  Grapefruit Juice
14                      Orange Juice
15                      Orange Juice
16     Orange Juice, 1/2oz Grenadine
17                   Cranberry Juice
18      1/2 Cranberry 1/2 Grapefruit
19       1/2 Cranberry 1/2 Pineapple
20          1/2 Cranberry 1/2 Orange
21          1/2 Cranberry 1/2 Orange
22                      Orange Juice
23                      Orange Juice
24            2oz Orange Juice, Soda
25                      Orange Juice
26                      Orange Juice
2

In [335]:
#-------------------Remove measurement text
# # 1/2, 1/2oz, 1oz, 1-, 2 cups
p = "(\d-\d)?(\d-\d/\d)?(\d/\d)?(/)?(\doz)?(\d drops)?(\d cup)?(\d can)?(cans)?(cups)?(z)?(oz)?"
pattern = re.compile(p)

# # remove measurement text from strings
df['Mixes'] = df['Mixes'].map(lambda x: re.sub(pattern, '', x))

In [336]:
df.Mixes[30:90]

30                                                Cream
31                                 Orange Juice,  Cream
32                                                Cream
33                                                Cream
34                                    Grenadine,  Cream
35                                 Orange Juice,  Cream
36                                                Cream
37                                          Cream, Soda
38                                                     
39                                                     
40                                                     
41                                                     
42                                                     
43                                                     
44                                                     
45                                                     
46                                                     
47                                              

In [337]:
df['Mixes'] = df.Mixes.apply(lambda x: x.strip(','))

In [340]:
# Remove undesired words and phrases
remove = ['drop', 'cups','can','cans','/zoz','on', 'top', 'no', 'splash', 'with', 's']
df['Mixes'] = df['Mixes'].apply(lambda x: ' '.join([i for i in x.lower().split() if i not in remove]))

In [341]:
df.Mixes

0                    gingerale
1                         soda
2                          7up
3                        water
4                         coke
5                         coke
6                        tonic
7               soda gingerale
8                         soda
9                         soda
10                tomato juice
11                tomato juice
12            grapefruit juice
13            grapefruit juice
14                orange juice
15                orange juice
16     orange juice, grenadine
17             cranberry juice
18        cranberry grapefruit
19         cranberry pineapple
20            cranberry orange
21            cranberry orange
22                orange juice
23                orange juice
24          orange juice, soda
25                orange juice
26                orange juice
27                orange juice
28                       cream
29                       cream
                ...           
101                           
102     

### Join Main Alcohol, Other Alcohols, and Mixes columns into single list

In [342]:
df[['Main Alcohol','Other Alcohols','Mixes']].head(20)

,Main Alcohol,Other Alcohols,Mixes
0,bourbon,,gingerale
1,scotch,,soda
2,seagram 7,,7up
3,whiskey,,water
4,light rum,,coke
5,light rum,,coke
6,gin,,tonic
7,bourbon,,soda gingerale
8,gin,,soda
9,bourbon,,soda


In [343]:
ma = df["Main Alcohol"].str.split(",", expand = True) 
oa = df["Other Alcohols"].str.split(",", expand = True)
m = df["Mixes"].str.split(",", expand = True)

In [344]:
new = pd.concat([ma, oa, m], axis=1).fillna('')
new['combined'] = new.values.tolist()
new['combined'] = new.combined.map(lambda x: list(filter(None, x)))

### Clean 'Garnish' columns

In [289]:
import re
# Remove 1/2, 1oz, 1/2 oz and similar
p1 = "(\d/)?(\d+oz)"
pattern1 = re.compile(p1)
# pattern.search(df['Mixes'][26]) # 18, 26
# <re.Match object; span=(14, 19), match='1/2oz'>
# <re.Match object; span=(0, 3), match='2oz'>
# Remove 1/2 and similar
p2 = "(\d/\d)"
pattern2 = re.compile(p2)
# pattern.search(df['Mixes'][8]) # 8
# <re.Match object; span=(0, 3), match='1/2'>
p3 = "(\d-)"
pattern3 = re.compile(p3)
# pattern.search(df['Mixes'][8]) # 8
# <re.Match object; span=(0, 3), match='1/2'>
p4 = "(\d dashes)"
pattern4 = re.compile(p4)
p5 = "(\d drops)"
pattern5 = re.compile(p5)

# # remove measurement text from strings
df['Garnish'] = df['Garnish'].map(lambda x: re.sub(pattern1, '', x))
# # remove measurement text from strings
df['Garnish'] = df['Garnish'].map(lambda x: re.sub(pattern2, ',', x))
# # remove measurement text from strings
df['Garnish'] = df['Garnish'].map(lambda x: re.sub(pattern3, ',', x))
df['Garnish'] = df['Garnish'].map(lambda x: re.sub(pattern4, '', x))
df['Garnish'] = df['Garnish'].map(lambda x: re.sub(pattern5, '', x.lower()))

In [290]:
df.Garnish

0                                                       
1                                                       
2                                                       
3                                                       
4                                                       
5                          squeeze and garnish with lime
6                                                   lime
7                                                       
8                                                   lime
9       simple syrup or sugar,  bitters, 1 orangle sl...
10      salt,  pepper,  celery salt,  worscestershire...
11      salt,  pepper,  celery salt,  worscestershire...
12                                                      
13                                                      
14                                                      
15                                                      
16                                                cherry
17                         (lim

In [291]:
final = new['combined'].apply(pd.Series)
final.columns = ['Liquor','Ingredient1','Ingredient2','Ingredient3','Ingredient4','Ingredient5','Ingredient6','Ingredient7']

In [292]:
final['Garnish'] = df.Garnish

In [293]:
final.fillna('', inplace=True)

In [294]:
final.head()

,Liquor,Ingredient1,Ingredient2,Ingredient3,Ingredient4,Ingredient5,Ingredient6,Ingredient7,Garnish
0,bourbon,gingerale,,,,,,,
1,scotch,soda,,,,,,,
2,seagram 7,7up,,,,,,,
3,whiskey,water,,,,,,,
4,light rum,coke,,,,,,,


In [295]:
final.to_csv("cleanBartenderExaminer.csv")

## Four Questions

In [51]:
import pandas as pd

In [52]:
final = pd.read_csv("cleanBartenderExaminer.csv", index_col=0).fillna('')
final.head()

,Liquor,Ingredient1,Ingredient2,Ingredient3,Ingredient4,Ingredient5,Ingredient6,Ingredient7,Garnish
0,bourbon,gingerale,,,,,,,
1,scotch,soda,,,,,,,
2,seagram 7,7up,,,,,,,
3,whiskey,water,,,,,,,
4,light rum,coke,,,,,,,


In [3]:
final['Liquor'].unique()

array(['bourbon', 'scotch', 'seagram 7', 'whiskey', 'light rum', 'gin',
       'vodka', 'tequila', 'sloe gin', 'amaretto', 'peach schnapps',
       'brandy', 'galliano', 'creme de banana', 'creme de almond',
       'green creme de menthe', 'kahlua', 'vermouths', 'southern comfort',
       'cognac', 'myers rum', 'when applicable', 'several',
       'jack daniels', 'sweet and sour', '2 rum', '2 bourbon',
       'malibu rum', 'yukon jack', 'sambuca', 'baileys',
       'butterscotch schnapps', 'cinnamon schnapps',
       'lemon flavoured vodka', 'vanilla flavoured vodka', 'jagermeister'],
      dtype=object)

In [31]:
# Liquor - whiskey, vodka, tequila, rum, gin, other
final['Liquor'].value_counts().to_frame()
# Mapping:
liquor_dict = {'whiskey':'whiskey',
               'whisky':'whiskey',
               'bourbon':'whiskey',
               'brandy':'whiskey',
               'jack daniels':'whiskey',
               'southern comfort':'whiskey',
               'brandy':'whiskey',
               'yukon jack':'whiskey',
               'seagram':'whiskey',
               'jagermeister':'whiskey',
               'vodka':'vodka',
               'gin':'vodka',
               'vermouth':'vodka',
               'tequila':'tequila',
               'sambuca':'tequila',
               'galliano':'tequila',
               'sweet':'tequila',
               'rum':'rum',
               'schnapps':'rum',
               'amaretto':'rum',
               'creme':'rum',
               'kahlua':'rum',
               'cognac':'rum',
               'baileys':'rum',
               'scotch':'whiskey',}

def whichliquor(x):
    temp = [v for k, v in liquor_dict.items() if k in x]
    if temp:
        return liquor_dict[temp[0]]
    else:
        return 'other'
    
final['a'] = final.Liquor.apply(whichliquor)

In [58]:
# Season
import random
seasons = ['fall', 'winter', 'spring', 'summer']

winter_words = ['cocoa','warm', 'tea', 'hot', 'peppermint', 'chocolate', 'creme', 'half', 'kahlua', 'milk'] 

summer_words = ['lemon','orange', 'lime', 'mint', 'sugar', 'cucumber', 'peach', 'pineapple', 'grapefruit', 'vermouth', 'triple sec',' ice'] 

spring_words = ['berry','grape', 'melon', 'soda', 'mango', 'sour', 'grenadine', '7-up'] 

fall_words = ['spice', 'orange', 'pumpkin', 'cream', 'apple', 'clove', 'ginger', 'syrup', 'nutmeg','coffee'] 

season_dict = {'winter': winter_words, 'summer': summer_words, 'spring': spring_words,'fall': fall_words}

season_dict2 = {}
for k,v in season_dict.items():
    for x in v:
        season_dict2[x.strip()] = k
        
def whichseason(x):
    temp = [v for k, v in season_dict2.items() if k in x]
    if temp:
        return temp[0]
    else:
        return random.choice(seasons)

final['b'] = final.apply(lambda x: ' '.join(x), axis=1).apply(whichseason)

In [59]:
# Music (#ingredients)
def whichmusic(x):
    if x <= 1:
        return "Hot club hits"
    if x == 2:
        return "Big chart-toppers"
    if x == 3:
        return "Probably rock"
    if x >= 4:
        return "Some Sinatra"
    
final['c'] = final[['Ingredient1', 'Ingredient2', 'Ingredient3', 'Ingredient4',
       'Ingredient5', 'Ingredient6', 'Ingredient7']].apply(lambda x: (','.join(x).split(',')), axis=1)\
                                                    .apply(lambda x: len(list(filter(None, x))))\
                                                    .apply(whichmusic)

In [89]:
# Flavor Profile
Frozen =  [' ice', 'ice ', 'coke', 'cola', '7-up', 'soda'] 
Fruity = ['orange', 'apple', 'peach', 'pineapple', 'grapefruit','berry', 'grape', 'melon',  'mango'] 
Hot = ['creme', 'cream'] 
Sour = ['sour', 'lime', 'lemon'] 
Spicy = ['whisky', 'whiskey', 'spice', 'ginger', 'cider', 'pepper','rum','tabasco','spicy', 'bloody', 'dry vermouth', 'scotch', 'southern comfort'] 
Sweet = ['liquer','liqueur', 'sweet', 'triple sec', 'amaretto', 'grenadine', 'syrup', 'sugar', 'midori', 'milk', 'half', 'grand'] 

flavor_dict = {'frozen': Frozen, 'fruity': Fruity, 'hot': Hot,'sour': Sour,'spicy': Spicy,'sweet':Sweet}
flavors = list(flavor_dict.keys())

flavor_dict2 = {}
for k,v in flavor_dict.items():
    for x in v:
        flavor_dict2[x.strip()] = k
        
def whichflavor(x):
    temp = [v for k, v in flavor_dict2.items() if k in x]
    if temp:
        return temp[0]
    else:
        return random.choice(flavors)

final['d'] = final[['Liquor', 'Ingredient1', 'Ingredient2', 'Ingredient3', 'Ingredient4',
       'Ingredient5', 'Ingredient6', 'Ingredient7', 'Garnish']].apply(lambda x: ' '.join(x), axis=1).apply(whichflavor)

In [90]:
final.head()

,Liquor,Ingredient1,Ingredient2,Ingredient3,Ingredient4,Ingredient5,Ingredient6,Ingredient7,Garnish,a,b,c,d
0,bourbon,gingerale,,,,,,,,whiskey,fall,Hot club hits,spicy
1,scotch,soda,,,,,,,,whiskey,spring,Hot club hits,frozen
2,seagram 7,7up,,,,,,,,whiskey,fall,Hot club hits,hot
3,whiskey,water,,,,,,,,whiskey,winter,Hot club hits,spicy
4,light rum,coke,,,,,,,,rum,spring,Hot club hits,frozen


In [91]:
final.to_csv("BartenderExaminer_Final.csv")